<a href="https://colab.research.google.com/github/RamiKassouf/MachineLearning-ObjectDetectionAPI/blob/main/RCNN_to_ONNX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 94.3 MB/s eta 0:00:00


In [ ]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import onnx
import torchvision.transforms as transforms
import PIL.Image

In [ ]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
def get_object_detection_model(num_classes = 3,
                               feature_extraction = True):
    """
    Inputs
        num_classes: int
            Number of classes to predict. Must include the
            background which is class 0 by definition!
        feature_extraction: bool
            Flag indicating whether to freeze the pre-trained
            weights. If set to True the pre-trained weights will be
            frozen and not be updated during.
    Returns
        model: FasterRCNN
    """
    # Load the pretrained faster r-cnn model.
    model = fasterrcnn_resnet50_fpn(pretrained = True)
    # If True, the pre-trained weights will be frozen.
    if feature_extraction == True:
        for p in model.parameters():
            p.requires_grad = False
    in_feats = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_feats,
                                                   num_classes)
    return model

In [ ]:
checkpoint_path = '/content/model_weights5.pt'
new_model = get_object_detection_model(num_classes = 3,
                        feature_extraction = False)
new_model.load_state_dict(torch.load(checkpoint_path,map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
new_model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
output_onnx = '/content/faster_rcnn_dolly.onnx'
print("==> Exporting model to ONNX format at '{}'".format(output_onnx))
input_names = ["input0"]
output_names = ["output0"]
img = PIL.Image.open("/content/drive/MyDrive/sample_dataset/images/training_images_take_7_280.png").convert("RGB")
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

# Apply the transformation to the image
tensor = transform(img)


# Add an extra dimension to the tensor to represent the batch size
tensor = tensor.unsqueeze(0)
# tensor=tensor.to(torch.device('cuda'))

torch_out = torch.onnx._export(new_model, tensor, output_onnx, export_params=True, verbose=False)

==> Exporting model to ONNX format at '/content/faster_rcnn_dolly.onnx'


<ipython-input-10-5764fe778a97>:19: FutureWarning: 'torch.onnx._export' is deprecated in version 1.12.0 and will be removed in 2.0. Please use `torch.onnx.export` instead.
  torch_out = torch.onnx._export(new_model, tensor, output_onnx, export_params=True, verbose=False)
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:3912: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  (torch.floor((input.size(i + 2).float() * torch.tensor(scale_factors[i], dtype=torch.float32)).float()))
/usr/local/lib/python3.10/dist-packages/torchvision/ops/boxes.py:157: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_x = torch.min(boxes_x, torch.tensor(width, dtype=boxes.dtype, device=boxes.device)

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
# # Verify that the model can be loaded from the ONNX file
onnx_model = onnx.load('/content/faster_rcnn_dolly.onnx')
onnx.checker.check_model(onnx_model)

In [ ]:
import onnxruntime as ort
import numpy as np
sess_options = ort.SessionOptions()
# Below is for optimizing performance
sess_options.intra_op_num_threads = 24
sess_options.execution_mode = ort.ExecutionMode.ORT_PARALLEL
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
ort_session = ort.InferenceSession("faster_rcnn_dolly.onnx", sess_options=sess_options)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(tensor)}
ort_outs = ort_session.run(None, ort_inputs)

In [ ]:
import onnxruntime.backend as backend




rep = backend.prepare(onnx_model, 'CPU')

outputs= rep.run(to_numpy(tensor))
print(outputs)

[array([[253.94157 , 312.50266 , 275.21127 , 368.6288  ],
       [191.23961 , 162.6227  , 453.6065  , 373.64035 ],
       [387.2236  , 314.87988 , 412.0344  , 363.29092 ],
       [236.82425 , 312.18924 , 278.31042 , 370.0172  ],
       [217.43071 , 167.11778 , 442.30444 , 369.13394 ],
       [184.73203 ,  59.692116, 560.33887 , 414.93625 ]], dtype=float32), array([2, 1, 2, 2, 2, 1], dtype=int64), array([0.43515712, 0.3581671 , 0.2285907 , 0.09497029, 0.06296045,
       0.06108709], dtype=float32)]
